In [1]:
%load_ext rpy2.ipython

In [2]:
%%R -o gene2name
suppressMessages(suppressWarnings(library(tidyverse)))

GTF = '~/genomes/hg38/gencode.v34/gencode.v34.annotation.gtf'

gtf <- rtracklayer::import(GTF)
gene2name <- gtf[gtf$type == "gene"] %>% data.frame %>% column_to_rownames('gene_id') %>% dplyr::select('gene_name')

In [3]:
import pandas as pd
import numpy as np

> ### event-level analysis
you can also do an event-level analysis
where you use the exon-level results
but as genefile use ChIP-seq data intersected with ex1-ex2-ex3 event (basically upstrem and downstream exon included with the intron in the middle)

> for exon-level analysus
`JunctionSeqallGenes.results.txt.gz`
has the exon coordinates `chr	start	end	strand`
also has the transcripts that exon junction is in transcripts
you can go to gff flatfile and find exons upstream and downstream of the featyre at hand

In [6]:
def read_results_table(PATH):
    '''event-level analysis
    '''
    table = pd.read_csv(PATH,sep='\t').drop("featureID",axis=1).dropna(subset=['expr_case']).dropna(axis=1)
    # .drop("featureID",axis=1)
    table = pd.concat([
        gene2name.loc[table.geneID,:].reset_index(drop=True),
        table.reset_index(drop=True)
    ],axis=1).set_index(["geneID","gene_name","countbinID"])
    
    table=table[[i for i in table if len(set(table[i]))>1]]

    return table

In [7]:
sigtable = read_results_table('jscs/sigGenes.results.txt.gz')

In [8]:
sigtable

baseMean      baseVar  \
geneID             gene_name countbinID                             
ENSG00000179051.14 RCC2      E001        1103.121233  7371.329806   
                             E002        1069.540708  3891.206624   
                             E003         504.215310  1435.184194   
                             E004         657.354475  2735.846681   
                             E005         426.280959  1394.518697   
...                                              ...          ...   
ENSG00000164934.14 DCAF13    J037         122.644326   419.186297   
                             J038          93.878885   475.129671   
                             J039         104.921183  1055.059573   
                             J040         128.404928   354.496535   
                             J041          95.359030   376.906665   

                                         dispBeforeSharing  dispFitted  \
geneID             gene_name countbinID                                  
ENSG00000179051.14 RCC2      E001                 0.002801    0.005524   
                             E002                 0.000835    0.005542   
                             E003                 0.001550    0.006213   
                             E004                 0.003226    0.005917   
                             E005                 0.001769    0.006445   
...                                                    ...         ...   
ENSG00000164934.14 DCAF13    J037                 0.012479    0.009482   
                             J038                 0.016042    0.010860   
                             J039                 0.044913    0.010242   
                             J040                 0.003578    0.009281   
                             J041                 0.002818    0.010769   

                                         dispersion    pvalue   padjust   chr  \
geneID             gene_name countbinID                                         
ENSG00000179051.14 RCC2      E001          0.003493  0.385198  0.891435  chr1   
                             E002          0.001938  0.915648  0.993577  chr1   
                             E003          0.002780  0.811970  0.982305  chr1   
                             E004          0.003946  0.357536  0.881314  chr1   
                             E005          0.003093  0.643564  0.956961  chr1   
...                                             ...       ...       ...   ...   
ENSG00000164934.14 DCAF13    J037          0.011426  0.716768  0.968589  chr8   
                             J038          0.014149  0.193297  0.790086  chr8   
                             J039          0.031468  0.153376  0.753985  chr8   
                             J040          0.005744  0.116156  0.706334  chr8   
                             J041          0.005833  0.013832  0.385439  chr8   

                                             start        end strand  \
geneID             gene_name countbinID                                
ENSG00000179051.14 RCC2      E001         17406759   17407838      -   
                             E002         17407838   17408554      -   
                             E003         17408554   17408675      -   
                             E004         17408675   17408979      -   
                             E005         17408979   17409046      -   
...                                            ...        ...    ...   
ENSG00000164934.14 DCAF13    J037        103430689  103432658      +   
                             J038        103432741  103435625      +   
                             J039        103435790  103440135      +   
                             J040        103440271  103441454      +   
                             J041        103441618  103442794      +   

                                                                               transcripts  \
geneID             gene_name countbinID                                                      
ENSG

In [9]:
len(set(sigtable.index.to_frame().geneID))

29

In [10]:
sigtable.head()

baseMean      baseVar  \
geneID             gene_name countbinID                             
ENSG00000179051.14 RCC2      E001        1103.121233  7371.329806   
                             E002        1069.540708  3891.206624   
                             E003         504.215310  1435.184194   
                             E004         657.354475  2735.846681   
                             E005         426.280959  1394.518697   

                                         dispBeforeSharing  dispFitted  \
geneID             gene_name countbinID                                  
ENSG00000179051.14 RCC2      E001                 0.002801    0.005524   
                             E002                 0.000835    0.005542   
                             E003                 0.001550    0.006213   
                             E004                 0.003226    0.005917   
                             E005                 0.001769    0.006445   

                                         dispersion    pvalue   padjust   chr  \
geneID             gene_name countbinID                                         
ENSG00000179051.14 RCC2      E001          0.003493  0.385198  0.891435  chr1   
                             E002          0.001938  0.915648  0.993577  chr1   
                             E003          0.002780  0.811970  0.982305  chr1   
                             E004          0.003946  0.357536  0.881314  chr1   
                             E005          0.003093  0.643564  0.956961  chr1   

                                            start       end strand  \
geneID             gene_name countbinID                              
ENSG00000179051.14 RCC2      E001        17406759  17407838      -   
                             E002        17407838  17408554      -   
                             E003        17408554  17408675      -   
                             E004        17408675  17408979      -   
                             E005        17408979  17409046      -   

                                                                               transcripts  \
geneID             gene_name countbinID                                                      
ENSG00000179051.14 RCC2      E001                                        ENST00000375436.9   
                             E002                      ENST00000375436.9;ENST00000474892.1   
                             E003                                        ENST00000375436.9   
                             E004                      ENST00000375436.9;ENST00000375433.3   
                             E005        ENST00000375436.9;ENST00000474892.1;ENST000003...   

                                         featureType  padjust_noFilter  \
geneID             gene_name countbinID                                  
ENSG00000179051.14 RCC2      E001        exonic_part               1.0   
                             E002        exonic_part               1.0   
                             E003        exonic_part               1.0   
                             E004        exonic_part               1.0   
                             E005        exonic_part               1.0   

                                         log2FC(case/ctrl)  \
geneID             gene_name countbinID                      
ENSG00000179051.14 RCC2      E001                 0.095052   
                             E002                 0.011321   
                             E003                -0.013476   
                             E004                -0.102690   
                             E005                 0.049613   

                                         log2FCvst(case/ctrl)    expr_ctrl  \
geneID             gene_name countbinID                                      
ENSG00000179051.14 RCC2      E001                    0.047508  1070.229083   
                             E002                    0.005658  1049.215055   
                             E003                   -0.00

In [11]:
alltable = read_results_table('jscs/allGenes.results.txt.gz')

In [12]:
len(set(alltable.index.to_frame().geneID))

8187

In [13]:
alltable.head()

baseMean      baseVar  \
geneID             gene_name countbinID                             
ENSG00000187634.12 SAMD11    E036          89.732602   172.300518   
                             E037         543.074952  5135.919936   
                             E038         366.493881  3350.628900   
                             E039         445.840223  4990.673608   
ENSG00000188976.11 NOC2L     E003        1003.356509  9395.312496   

                                         dispBeforeSharing  dispFitted  \
geneID             gene_name countbinID                                  
ENSG00000187634.12 SAMD11    E036                 0.000048    0.012074   
                             E037                 0.007980    0.006122   
                             E038                 0.007260    0.006690   
                             E039                 0.015188    0.006379   
ENSG00000188976.11 NOC2L     E003                 0.002240    0.005581   

                                         dispersion    pvalue   padjust   chr  \
geneID             gene_name countbinID                                         
ENSG00000187634.12 SAMD11    E036          0.004946  0.056284  0.588518  chr1   
                             E037          0.007066  0.083785  0.651989  chr1   
                             E038          0.007025  0.147051  0.747050  chr1   
                             E039          0.011414  0.008529  0.335733  chr1   
ENSG00000188976.11 NOC2L     E003          0.003064  0.186209  0.783452  chr1   

                                          start     end strand  \
geneID             gene_name countbinID                          
ENSG00000187634.12 SAMD11    E036        943907  944259      +   
                             E037        944259  944574      +   
                             E038        944574  944575      +   
                             E039        944575  944581      +   
ENSG00000188976.11 NOC2L     E003        944204  944800      -   

                                                                               transcripts  \
geneID             gene_name countbinID                                                      
ENSG00000187634.12 SAMD11    E036        ENST00000618779.4;ENST00000618323.4;ENST000006...   
                             E037        ENST00000618779.4;ENST00000618323.4;ENST000006...   
                             E038        ENST00000618779.4;ENST00000618323.4;ENST000006...   
                             E039        ENST00000618779.4;ENST00000618323.4;ENST000006...   
ENSG00000188976.11 NOC2L     E003        ENST00000327044.7;ENST00000483767.5;ENST000004...   

                                         featureType  padjust_noFilter  \
geneID             gene_name countbinID                                  
ENSG00000187634.12 SAMD11    E036        exonic_part          0.838738   
                             E037        exonic_part          0.898776   
                             E038        exonic_part          0.976673   
                             E039        exonic_part          0.548082   
ENSG00000188976.11 NOC2L     E003        exonic_part          1.000000   

                                         log2FC(case/ctrl)  \
geneID             gene_name countbinID                      
ENSG00000187634.12 SAMD11    E036                 0.332724   
                             E037                 0.348117   
                             E038                -0.249536   
                             E039                -0.534640   
ENSG00000188976.11 NOC2L     E003                 0.135332   

                                         log2FCvst(case/ctrl)    expr_ctrl  \
geneID             gene_name countbinID                                      
ENSG00000187634.12 SAMD11    E036                    0.166332    89.712308   
                             E037                    0.172860   564.768958   
                             E038                   -0.124596   390.001674   
       

___
> ### gives a gene-level file as well, you can use that. 
to run teiser
with ChIP-seq data intersected with genes as genefile

> this is the gene based file
`JunctionSeqsigGenes.genewiseResults.txt.gz`
genes listed here go to class ‘1’
and all the other expressed genes class ‘0’



In [14]:
def read_genewiseResults_table(PATH):
    '''event-level analysis
    '''
    table = pd.read_csv(PATH,sep='\t')#.drop("featureID",axis=1).dropna(subset=['expr_dox']).dropna(axis=1)
    table = table.set_index(["geneID","geneName"])

    return table

In [15]:
genewise_table = read_genewiseResults_table("jscs/sigGenes.genewiseResults.txt.gz")
genewise_genes = set(genewise_table.index.get_level_values(1))

In [16]:
genewise_table

,,chr,start,end,strand,baseMean,geneWisePadj,mostSigID,mostSigPadjust,numExons,numKnown,numNovel,exonsSig,knownSig,novelSig,numFeatures,numSig
geneID,geneName,,,,,,,,,,,,,,,,
ENSG00000179051.14,RCC2,chr1,17406759,17439677,-,4226.7,9.841564e-04,J032,5.370000e-04,19,13,0,1,1,0,19/13/0,1/1/0
ENSG00000053372.5,MRTO4,chr1,19251804,19260128,+,1654.2,5.966044e-03,E001,3.610000e-03,13,8,0,1,0,0,13/8/0,1/0/0
ENSG00000163399.16,ATP1A1,chr1,116372667,116410261,+,15877.3,1.311720e-02,E002,8.750000e-03,40,28,0,1,0,0,40/28/0,1/0/0
ENSG00000158716.9,DUSP23,chr1,159780931,159782543,+,544.5,8.004604e-03,E006,5.340000e-03,7,3,0,1,0,0,7/3/0,1/0/0
ENSG00000116750.13,UCHL5,chr1,193012249,193060080,-,1049.6,8.947062e-03,E027,6.060000e-03,37,20,0,2,0,0,37/20/0,2/0/0
ENSG00000175348.11,TMEM9B,chr11,8947201,8965011,-,404.2,1.324392e-02,E007,9.050000e-03,14,8,0,1,0,0,14/8/0,1/0/0
ENSG00000149136.9,SSRP1,chr11,57325985,57335892,-,5121.7,1.330073e-02,E032,8.980000e-03,33,20,0,1,0,0,33/20/0,1/0/0
ENSG00000167995.17,BEST1,chr11,61950062,61965515,+,49388.8,8.909568e-03,E030,6.060000e-03,30,15,0,1,0,0,30/15/0,1/0/0
ENSG00000089597.18,GANAB,chr11,62624825,62646726,-,7773.9,4.982960e-03,E048,2.930000e-03,59,33,0,2,1,0,59/33/0,2/1/0


In [19]:
def read_expression_table(PATH):
    '''
    '''
    table = pd.read_csv(PATH,sep='\t').dropna(subset=['expr_case']).dropna(axis=1)
    table["geneID"] = table.featureID.str.split(':').str[0]
    table["countbinID"] = table.featureID.str.split(':').str[1]
    
    table = pd.concat([
        gene2name.loc[table.geneID,:].reset_index(drop=True),
        table.drop("featureID",axis=1).reset_index(drop=True)
    ],axis=1).set_index(["geneID","gene_name","countbinID"])
    
    return table

In [20]:
exp_table = read_expression_table('jscs/allGenes.expression.data.txt.gz')

exp_table.index.get_level_values(1)
exp_genes = set(exp_table.index.get_level_values(1))

In [21]:
len(exp_genes)

8187

In [22]:
splice_binary_table = pd.DataFrame(
    {"spliced":np.zeros(len(exp_genes))},
    index=exp_genes, dtype=int
)

In [23]:
splice_binary_table

,spliced
APP,0
PIK3R4,0
ZFP36,0
SAV1,0
ZNF16,0
...,...
TSPAN1,0
COX7A2L,0
EOLA2,0
AC079305.2,0


In [24]:
splice_binary_table.loc[genewise_genes,:] = 1

In [43]:
splice_binary_table.to_csv("ChIP-seq-data/splice_binary_table.txt",sep='\t')

## ChIP-seq-data

    %%bash 
    GTF=~/genomes/hg38/gencode.v34/gencode.v34.annotation.gtf

    for f in ChIP-seq-data/bed_files/*.bed; do 
        o=${f/_IDX*/.txt}

        echo "================================================================"
        echo $o

        cat $GTF \
        | awk -v FS='\t' '$3=="gene"' \
        | bedtools intersect -wa -wb -a - -b $f \
        | cut -f 9,18 | uniq | awk -v FS='\t' '{print $1,$2}' \
        | awk -v FS=";" -v OFS="" '{gsub("\"",""); print $3}' \
        | awk -v FS=' ' -v OFS="" '{print $2}' > $o

    done


In [36]:
ls ChIP-seq-data/bed_files/*.txt

ChIP-seq-data/bed_files/GFP_1_DOX_pos.txt
ChIP-seq-data/bed_files/GFP_2_DOX_pos.txt
ChIP-seq-data/bed_files/s303_1_DOX_neg.txt
ChIP-seq-data/bed_files/s303_1_DOX_pos.txt
ChIP-seq-data/bed_files/s303_2_DOX_neg.txt
ChIP-seq-data/bed_files/s303_2_DOX_pos.txt
ChIP-seq-data/bed_files/sox9_1_DOX_neg.txt
ChIP-seq-data/bed_files/SOX9_1_DOX_pos.txt
ChIP-seq-data/bed_files/sox9_2_DOX_neg.txt
ChIP-seq-data/bed_files/SOX9_2_DOX_pos.txt


In [41]:
%%bash 
cat \
ChIP-seq-data/bed_files/SOX9_1_DOX_pos.txt \
ChIP-seq-data/bed_files/SOX9_2_DOX_pos.txt \
| uniq > ChIP-seq-data/SOX9_DOX_pos.txt

cat \
ChIP-seq-data/bed_files/sox9_1_DOX_neg.txt \
ChIP-seq-data/bed_files/sox9_2_DOX_neg.txt \
| uniq > ChIP-seq-data/SOX9_DOX_neg.txt

In [39]:
%%bash 
cat \
ChIP-seq-data/bed_files/GFP_1_DOX_pos.txt \
ChIP-seq-data/bed_files/GFP_2_DOX_pos.txt \
| uniq > ChIP-seq-data/GFP_DOX_pos.txt

In [40]:
%%bash 
cat \
ChIP-seq-data/bed_files/s303_1_DOX_neg.txt \
ChIP-seq-data/bed_files/s303_2_DOX_neg.txt \
| uniq > ChIP-seq-data/S303_DOX_neg.txt

cat \
ChIP-seq-data/bed_files/s303_1_DOX_pos.txt \
ChIP-seq-data/bed_files/s303_2_DOX_pos.txt \
| uniq > ChIP-seq-data/S303_DOX_pos.txt

In [45]:
ls ChIP-seq-data/*DOX*.txt

ChIP-seq-data/GFP_DOX_pos.txt   ChIP-seq-data/SOX9_DOX_neg.txt
ChIP-seq-data/S303_DOX_neg.txt  ChIP-seq-data/SOX9_DOX_pos.txt
ChIP-seq-data/S303_DOX_pos.txt


In [54]:
%%bash 
export TEISERDIR='/data_gilbert/home/aarab/Workflows/TEISERv1.1'
exp="splice_binary_table.txt"

cd ChIP-seq-data/
for gene in *DOX*.txt; do

    perl ${TEISERDIR}/run_mi_gene_list.pl \
        --expfile=$exp \
        --genefile=$gene \
        --exptype=discrete \
        --species=human \
        --doremovedups=0 \
        --doremoveextra=0 
        # --ebins=11 \

    perl /data_gilbert/home/aarab/Workflows/TEISERv1.1/Scripts/teiser_draw_matrix.pl \
        --pvmatrixfile=splice_binary_table.txt_GENESET/splice_binary_table.txt.matrix \
        --summaryfile=splice_binary_table.txt_GENESET/splice_binary_table.txt.summary \
        --expfile=splice_binary_table.txt_GENESET/splice_binary_table.txt \
        --quantized=1 \
        --colmap=/data_gilbert/home/aarab/Workflows/TEISERv1.1/Scripts/HEATMAPS/cmap_1.txt --order=0 --min=-3 --max=3 --cluster=5 

    out=${gene/.txt/}
    rm -rf $out
    mv ${exp}_GENESET $out
    cp $out/${exp}.summary.pdf ${out}.pdf
done
cd ../

The TEISERDIR environment variable is /data_gilbert/home/aarab/Workflows/TEISERv1.1
Mon Jan 24 20:13:31 PST 2022
Remove duplicates, create splice_binary_table.txt_GENESET/splice_binary_table.txt
step 2: seed optimization.
Expfile loaded: 8187 values...
Quantizing the input vector...Done
Number of clusters: 2
calculating the p-value matrix.
Allocating memory ... Done
doing stats: mi = 0.000001
pass = 1.000000
z = -0.078693
freq 0: 0.000245	2	8158	2	8187
freq 1: 0.000000	0	29	2	8187
step 7: drawing matrix.
Reading MI data ... Done.
Start drawing
0
Outputing EPS file splice_binary_table.txt_GENESET/splice_binary_table.txt.summary.eps
Convert to PDF splice_binary_table.txt_GENESET/splice_binary_table.txt.summary.pdf
ps2pdf -dEPSCrop -dAutoRotatePages=/None splice_binary_table.txt_GENESET/splice_binary_table.txt.summary.eps splice_binary_table.txt_GENESET/splice_binary_table.txt.summary.pdf
Finished.
Reading MI data ... Done.
Start drawing
0
Outputing EPS file splice_binary_table.txt_GENESE

Option suffix requires an argument
Option suffix requires an argument
Option suffix requires an argument
Option suffix requires an argument
Option suffix requires an argument


In [193]:
ls /data_gilbert/home/aarab/iPAGE/run_mi_gene_list.pl

/data_gilbert/home/aarab/iPAGE/run_mi_gene_list.pl


In [183]:
!echo ${TEISERDIR}

In [80]:
!date

Thu Jan 13 19:49:49 PST 2022
